In [1]:
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5.QtGui import *
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import sys

class UI(QMainWindow):
    def __init__(self):
        super(UI,self).__init__()
        
        #Data PreProcessing
        #Training Image processing
        self.train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True)
        self.training_set = self.train_datagen.flow_from_directory(
                    'training_set',target_size=(64,64),batch_size=32,class_mode='categorical')
        
        #Test image processing
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_set = test_datagen.flow_from_directory('test_set',target_size=(64,64),batch_size=32,
                                                    class_mode='categorical')
        
        #Building Model
        self.cnn = tf.keras.models.Sequential()

        #Building Convolution Layer
        #First layer
        self.cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu',input_shape=[64,64,3]))
        self.cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
        
        #Second Layer
        self.cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
        self.cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
        
        #Dropout
        self.cnn.add(tf.keras.layers.Dropout(0.5))
        #Flattening
        self.cnn.add(tf.keras.layers.Flatten())
        #hiddden layer
        self.cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))
        #Output Layer
        self.cnn.add(tf.keras.layers.Dense(units=5,activation='softmax'))
        #For compile results
        self.cnn.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
        
        self.cnn.fit(x=self.training_set,validation_data=test_set,epochs=1)
        
        
        # load the ui file
        uic.loadUi("interface_final.ui",self)
        # Define our widgets
        self.label =self.findChild(QLabel,"image_box")
        self.button =self.findChild(QPushButton,"browse")
        self.label1 =self.findChild(QLabel,"flower_label")

        # Click The Dropdown Box
        self.button.clicked.connect(self.clicker)
        # Show The App
        self.show()
    
    def clicker(self):
        fname=QFileDialog.getOpenFileName(self, 'Open file', r'"C:\Users\muhda\OneDrive - Universiti Teknologi MARA\SEM 3\CSC583\Flower_Recognition\Prediction"', 'Images (*.png, *.xmp *.jpg)')
        
        #Open image
        self.pixmap =QPixmap(fname[0])
        #add picture to image_box
        self.label.setPixmap(self.pixmap)

        test_image = tf.keras.utils.load_img(fname[0],target_size=(64,64))
        test_image = tf.keras.utils.img_to_array(test_image)
        test_image = np.expand_dims(test_image,axis=0)
        result = self.cnn.predict(test_image)
        self.training_set.class_indices
        
        if result[0][0]==1:
            self.label1.setText('These flower is  a Daisy')
        elif result[0][1]==1:
            self.label1.setText('These flower is  a Dandelion')
        elif result[0][2]==1:
            self.label1.setText('These flower is  a Rose')
        elif result[0][3]==1:
            self.label1.setText('These flower is  a SunFlower')
        elif result[0][4]==1:
            self.label1.setText('These flower is  a Tulip')
        else:
            self.label1.setText('can''t detect')

        

    #Preprocess New Image
    
# Initialize The App
app = QApplication(sys.argv)
UIWindow = UI()
app.exec_()

Found 4317 images belonging to 5 classes.
Found 4317 images belonging to 5 classes.
135/135 [==============================] - 119s 866ms/step - loss: 1.3058 - accuracy: 0.4466 - val_loss: 1.1818 - val_accuracy: 0.5272


0